<a href="https://colab.research.google.com/github/jingyuan0422/113-2-Programing-Language/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Use yfinance to download stock data
!pip install yfinance gspread oauth2client

In [44]:
"""
1. 資料準備
Download historical data of the top 30 U.S. companies by market capitalization from yfinance.
Data: closing prices, trading volume

2. 資料整理
Calculated the following daily metrics for each stock:
- Daily return (current day's close price / previous day's close price) - 1
- Rolling volatility (standard deviation of returns over a window of 20 days)
- Rolling Sharpe ratio (mean return divided by rolling volatility)
"""

import yfinance as yf
import pandas as pd
import numpy as np
import gspread
from google.colab import files
from oauth2client.service_account import ServiceAccountCredentials
import json


# Stock tickers
tickers = [
    "AAPL", "MSFT", "NVDA", "GOOG", "AMZN", "META", "BRK-B", "AVGO", "TSLA", "WMT",
    "LLY", "V", "JPM", "UNH", "MA", "XOM", "COST", "NFLX", "PG", "JNJ", "ORCL",
    "HD", "KO", "ABBV", "TMUS", "BAC", "PM", "CRM", "CVX", "ABT"
]

# date（from 2023/12/1 to 2024/12/31）
data = yf.download(tickers, start="2023-12-01", end="2024-12-31")

records = []

for ticker in tickers:
    try:
        price = data[("Close", ticker)].dropna() # closing prices
        volume = data[("Volume", ticker)].dropna() # trading volume
        daily_return = price.pct_change() # daily returns

        rolling_vol = daily_return.rolling(window=20).std() #    # rolling volatility
        rolling_sharpe = daily_return.rolling(window=20).mean() / rolling_vol # rolling Sharpe ratio

        # # Append data for each date to the records list
        for date in daily_return.index:
            if pd.isna(daily_return.loc[date]):
                continue
            records.append({
                "Ticker": ticker,
                "Date": date.strftime('%Y-%m-%d'),
                "Return": daily_return.loc[date],
                "Volatility": rolling_vol.loc[date],
                "Sharpe Ratio": rolling_sharpe.loc[date], # Sharp Ratio
                "Volume": volume.loc[date] if date in volume else np.nan
            })

    except Exception as e:
        print(f"{ticker} 有錯誤：{e}")

# Turn to Dataframe
df = pd.DataFrame(records)

# Filter out the data from 2023/12/1 to 2023/12/31
df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Date'] > '2023-12-31']

[*********************100%***********************]  30 of 30 completed


In [45]:
df

,Ticker,Date,Return,Volatility,Sharpe Ratio,Volume
19,AAPL,2024-01-02,-0.035787,0.012075,-0.117194,82488700
20,AAPL,2024-01-03,-0.007488,0.012014,-0.109565,58414500
21,AAPL,2024-01-04,-0.012700,0.011036,-0.272246,71983600
22,AAPL,2024-01-05,-0.004013,0.011021,-0.265024,62303300
23,AAPL,2024-01-08,0.024175,0.012272,-0.180816,59144500
...,...,...,...,...,...,...
8095,ABT,2024-12-23,0.000700,0.007547,-0.193255,4028200
8096,ABT,2024-12-24,0.003937,0.007575,-0.187306,1480000
8097,ABT,2024-12-26,0.004444,0.007687,-0.146299,2960500
8098,ABT,2024-12-27,-0.002429,0.007356,-0.226449,2829500


In [46]:
# Authorize Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [47]:
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o-E1qrYqv525Jcm2JH3AHgQN_bDAJHlYyRcoOY0--xA/edit?gid=0#gid=0')

from gspread_dataframe import set_with_dataframe

# Save data to sheet 1
worksheet = gsheets.get_worksheet(0)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, df)
print("完成，已成功將 data 儲存至 Google Sheets!")

完成，已成功將 data 儲存至 Google Sheets!


In [50]:
"""EDA 1
Average metrics for each stock
To identify which stocks had higher average returns, greater risk, and better Sharpe.
"""

# Grouped by ticker and calculated average metrics.
df_mean = df.groupby("Ticker")[["Return", "Volatility", "Sharpe Ratio"]].mean().reset_index()
df_mean.columns = ["Ticker", "Avg Return", "Avg Volatility", "Avg Sharpe Ratio"]

import plotly.express as px

# Bar chart sorted by average return
fig_return = px.bar(df_mean.sort_values("Avg Return", ascending=True),
                    x="Avg Return", y="Ticker", orientation="h",
                    title="📈 Avg Return",
                    labels={"Avg Return": "Average Return", "Ticker": "Stock Ticker"},
                    color="Avg Return",
                    color_continuous_scale="Viridis")
fig_return.show()

# Bar chart sorted by average volatility, with the largest at the top
fig_vol = px.bar(df_mean.sort_values("Avg Volatility", ascending=False),
                 x="Avg Volatility", y="Ticker", orientation="h",
                 title="📊 Avg Volatility",
                 labels={"Avg Volatility": "Average Volatility", "Ticker": "Stock Ticker"},
                 color="Avg Volatility",
                 color_continuous_scale="Viridis")
fig_vol.show()

# Bar chart sorted by average Sharpe ratio
fig_sharpe = px.bar(df_mean.sort_values("Avg Sharpe Ratio", ascending=True),
                    x="Avg Sharpe Ratio", y="Ticker", orientation="h",
                    title="⚖️ Avg Sharpe Ratio",
                    labels={"Avg Sharpe Ratio": "Average Sharpe Ratio", "Ticker": "Stock Ticker"},
                    color="Avg Sharpe Ratio",
                    color_continuous_scale="Viridis")
fig_sharpe.show()

# Investment Style Overview: Return vs Volatility (Bubble size = Sharpe Ratio)
fig_bubble = px.scatter(df_mean,
                        x="Avg Volatility",
                        y="Avg Return",
                        size=abs(df_mean["Avg Sharpe Ratio"]),  # Take the absolute value of Avg Sharpe Ratio for size
                        color="Ticker",
                        title="🎯 Investment Style Overview: Return vs Volatility (Bubble size = Sharpe Ratio)",
                        hover_name="Ticker",
                        size_max=40)
fig_bubble.show()

In [51]:
"""EDA 2
Distribution of Indicators for Each Stock (Boxplot)
Compare the return volatility across different stocks
"""

# Boxplot showing the distribution of returns for each stock
fig_return = px.box(df,
                    x="Ticker",
                    y="Return",
                    title="📈 Return Distribution by Stock",
                    labels={"Return": "Daily Return", "Ticker": "Stock Ticker"},
                    points="all")  # Display all points (to observe outliers)
fig_return.show()

# Boxplot showing the distribution of Sharpe Ratio for each stock
fig_sharpe = px.box(df,
                    x="Ticker",
                    y="Sharpe Ratio",
                    title="📊 Sharpe Ratio Distribution by Stock",
                    labels={"Sharpe Ratio": "Sharpe Ratio", "Ticker": "Stock Ticker"},
                    points="all")  # Display all points (to observe outliers)
fig_sharpe.show()

In [53]:
"""EDA 3
Distribution of Indicators for Each Stock (Boxplot)
Compare the return volatility across different stocks
"""

# Select a few representative stocks
tickers_to_plot = ["AAPL", "TSLA", "NVDA", # High return, high volatility stocks
                   "KO", "PG", "JNJ"] # Stable return, low volatility stocks
df_filtered = df[df['Ticker'].isin(tickers_to_plot)]

# Time Series Plot for returns
fig_return = px.line(df_filtered,
                     x="Date",
                     y="Return",
                     color="Ticker",
                     title="Time Series of Return for Selected Stocks",
                     labels={"Date": "Date", "Return": "Daily Return", "Ticker": "Stock Ticker"})
fig_return.show()

# Time Series Plot for returns volatility
fig_volatility = px.line(df_filtered,
                         x="Date",
                         y="Volatility",
                         color="Ticker",
                         title="Time Series of Volatility for Selected Stocks",
                         labels={"Date": "Date", "Volatility": "Volatility", "Ticker": "Stock Ticker"})
fig_volatility.show()

# Time Series Plot for returns Sharpe Ratio
fig_sharpe = px.line(df_filtered,
                     x="Date",
                     y="Sharpe Ratio",
                     color="Ticker",
                     title="Time Series of Sharpe Ratio for Selected Stocks",
                     labels={"Date": "Date", "Sharpe Ratio": "Sharpe Ratio", "Ticker": "Stock Ticker"})
fig_sharpe.show()

In [60]:
"""EDA 4
Trading Volume Analysis
Check the relationship between activity and return
"""

import plotly.express as px

# Scatter Plot of Trading Volume vs. Return for All Data
fig = px.scatter(df,
                 x="Volume",
                 y="Return",
                 color="Ticker",
                 title="Volume vs. Daily Return (All Stocks)",
                 labels={
                     "Volume": "Trading Volume",
                     "Return": "Daily Return"
                 },
                 opacity=0.6)
fig.show()

# Select 5 representative stocks by trading volume to plot their volume time series
selected_tickers = ["NVDA", "TSLA", "AAPL", "AMZN", "AVGO"]
df_volume = df[df["Ticker"].isin(selected_tickers)]

# Line chart
fig = px.line(df_volume,
              x="Date",
              y="Volume",
              color="Ticker",
              title="Trading Volume Over Time (Selected Stocks)",
              labels={
                  "Date": "Date",
                  "Volume": "Trading Volume",
                  "Ticker": "Stock Ticker"
              })
fig.show()

In [67]:
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Calculate the behavioral data for each stock (average Return, Volatility, Sharpe Ratio, and Volume
df_mean = df.groupby("Ticker")[["Return", "Volatility", "Sharpe Ratio", "Volume"]].mean().reset_index()

# Standardize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_mean[['Return', 'Volatility', 'Sharpe Ratio', 'Volume']])

# KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
df_mean['cluster'] = kmeans.fit_predict(scaled_features)

# PCA
pca = PCA(n_components=2)
components = pca.fit_transform(scaled_features)
df_mean['pca1'] = components[:, 0]
df_mean['pca2'] = components[:, 1]

# Visualization
fig = px.scatter(
    df_mean,
    x='pca1',
    y='pca2',
    color='cluster',
    title="Stock Behavior Clustering Results",
    labels={'pca1': 'Principal Component 1', 'pca2': 'Principal Component 2', 'cluster': 'Cluster Group'},
    hover_data=['Ticker', 'Return', 'Volatility', 'Sharpe Ratio', 'Volume']
)

# Update chart layout
fig.update_layout(
    xaxis_title="Principal Component 1",
    yaxis_title="Principal Component 2",
    legend_title="Group",
    title_x=0.5,  # Center the chart title
    height=600,
    width=800
)
# 顯示圖表
fig.show()

# Cluster Definitions
cluster_definitions = {
    0: "Cluster 0: High return, high volatility, low Sharpe ratio - Potentially high-risk stocks, suitable for investors seeking high returns.",
    1: "Cluster 1: Low return, low volatility, high Sharpe ratio - Stable, low-risk investment options, suitable for risk-averse investors.",
    2: "Cluster 2: Medium return, high volatility, low Sharpe ratio - Potentially high-risk stocks with greater return potential.",
    3: "Cluster 3: High return, low volatility, high Sharpe ratio - Stable stocks with higher return potential, suitable for long-term conservative investors."
}

# Print Cluster
for cluster_id, definition in cluster_definitions.items():
    print(definition)

Cluster 0: High return, high volatility, low Sharpe ratio - Potentially high-risk stocks, suitable for investors seeking high returns.
Cluster 1: Low return, low volatility, high Sharpe ratio - Stable, low-risk investment options, suitable for risk-averse investors.
Cluster 2: Medium return, high volatility, low Sharpe ratio - Potentially high-risk stocks with greater return potential.
Cluster 3: High return, low volatility, high Sharpe ratio - Stable stocks with higher return potential, suitable for long-term conservative investors.
